# Generate data with MRI background

Note that data generation is slightly different than in the other notebooks; this is an older version. However, as the MRI data is not available anymore in the preprocessed form, the updated version could not be tested with this background. 

It would be strongly recommended to generate the MRI data with the other notebook or to harmonize the two implementations. One simple way to do this is simply generating a numpy array with all MRI slices and passing it as a background in the other notebook and simply treating is as a noise condition, as also described in the manuscript.

NOTE: since the MRI data is privacy-sensitive, the outputs have been cleared from this notebook. The data is from the Human Connectome project and has been preprocessed according to the manuscript. 

In [ ]:
import pandas as pd
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
plt.rc('image', cmap='gray')

from copy import deepcopy

import time, datetime 

import h5py
import pickle as pkl

from functools import reduce
import operator

from sklearn.preprocessing import minmax_scale

import scipy.signal

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
working_dir = '/ritter/share/data/Connectome/connectome_BIDS_processed/'
data_dir = '/ritter/share/projects/Methods/Budding_interpret_ground_truth/data/hamming_single_lesion_lateral/'

In [ ]:
path = os.path.join(working_dir, 'participants.csv')

In [ ]:
data_parts = pd.read_csv(path)
data_parts['subj_idx'] = np.arange(len(data_parts))

In [ ]:
training_set = data_parts[data_parts['split'] == 'train']
validation_set = data_parts[data_parts['split'] == 'val']
holdout_set = data_parts[data_parts['split'] == 'holdout']

In [ ]:
training_set.head()

## Define functions for lesions

In [ ]:
path_data = os.path.join(working_dir, training_set['path'][407])
scan = nib.load(path_data)
struct_arr = scan.get_data().astype(np.float32)
struct_arr = struct_arr[10:150, :, :135]
slices = [struct_arr[:, :, i] for i in range(struct_arr.shape[2])]

In [ ]:
fig = plt.figure(figsize = (20, 8))
for i in range(10): 
    plt.subplot(1, 10, i + 1)
    plt.imshow(slices[i])

In [ ]:
def get_coordinates(struct_arr, num_lesions = 1, mask = None):
    lesion_img = deepcopy(struct_arr)
    
    list_coords = []
    
    if mask is not None: 
        struct_arr = mask * struct_arr
    
    for i in range(num_lesions):
        lesion = 'none'
        while lesion == 'none': 
            coord_1 = np.random.randint(0, struct_arr.shape[0])
            coord_2 = np.random.randint(0, struct_arr.shape[1])
            if struct_arr[coord_1, coord_2] != 0: 
                lesion = 'success'
                
        list_coords.append((coord_1, coord_2))
                
    return list_coords

def gaus2d(x=0, y=0, mx=0, my=0, sx=2, sy=2):
    return 1. / (2. * np.pi * sx * sy) * np.exp(-((x - mx)**2. / (2. * sx**2.) + (y - my)**2. / (2. * sy**2.)))

def _nd_window(data, filter_function, **kwargs):
    """
    Performs an in-place windowing on N-dimensional spatial-domain data.
    This is done to mitigate boundary effects in the FFT.

    Parameters
    ----------
    data : ndarray
           Input data to be windowed, modified in place.
    filter_function : 1D window generation function
           Function should accept one argument: the window length.
           Example: scipy.signal.hamming
    """
    for axis, axis_size in enumerate(data.shape):
        # set up shape for numpy broadcasting
        filter_shape = [1, ] * data.ndim
        filter_shape[axis] = axis_size
        window = filter_function(axis_size, **kwargs).reshape(filter_shape)
        # scale the window intensities to maintain image intensity
        np.power(window, (1.0/data.ndim), out=window)
        data *= window
        
    return data

def create_simple_lesions(struct_arr, list_coords, size, intensities = 0, shape = 'square', lesion_type = 'opaque', window_type = None): 
    lesion_img = deepcopy(struct_arr)
    ground_truth = np.zeros((struct_arr.shape[0], struct_arr.shape[1]))
    
    for i, coord in enumerate(list_coords):        
        if shape[i] == 'square':
            intensity = intensities[i]
            coords = [coord[0]- size, coord[0] + size, coord[1] - size, coord[1] + size]
            corr_coord = [0 if i < 0 else i for i in coords]
            
            if lesion_type[i] == 'opaque':
                lesion_img[corr_coord[0]:corr_coord[1], corr_coord[2]:corr_coord[3]] = intensity
                
            elif lesion_type[i] == 'reduced': 
                masked_area = struct_arr[corr_coord[0]:corr_coord[1], corr_coord[2]:corr_coord[3]]
                lesion_img[corr_coord[0]:corr_coord[1], corr_coord[2]:corr_coord[3]] = intensity * masked_area
                
            ground_truth[corr_coord[0]:corr_coord[1], corr_coord[2]:corr_coord[3]] = 1
                
        elif shape[i] == 'circular':
            intensity = intensities[i]
            h = struct_arr.shape[0]
            l = struct_arr.shape[1]
            
            y,x = np.ogrid[-coord[0]:h-coord[0], -coord[1]:l-coord[1]]
            mask = x*x + y*y <= size*size

            if lesion_type[i] == 'opaque': 
                new_value = intensity
            elif lesion_type[i] == 'reduced': 
                masked_area = struct_arr[mask]
                new_value = intensity * masked_area
                
            lesion_img[mask] = new_value
            
            ground_truth[mask] = 1
            
        elif shape[i] == 'gaussian': 
            x = np.linspace(0, struct_arr.shape[1], struct_arr.shape[1])
            y = np.linspace(0, struct_arr.shape[0], struct_arr.shape[0])
            x, y = np.meshgrid(x, y) # get 2D variables instead of 1D
            z = gaus2d(x, y, mx = coord[1], my = coord[0], sx = size, sy = size)
            z /= z.max()
            
            ground_truth += z
            
            z = (z * -1) + 1
            z = minmax_scale(z.flatten(), feature_range = (intensities[i], 1)).reshape(struct_arr.shape[0], struct_arr.shape[1])
            
            lesion_img = z * lesion_img
            
        elif shape[i] == 'window': 
            window_fnc = window_type
            
            masked_area = np.ones((2*size, 2*size))
            lesion = _nd_window(masked_area, filter_function = window_fnc)
            padded_img = np.ones((struct_arr.shape[0] + 2 * size, struct_arr.shape[1] + 2 * size))
                        
            padded_img[coord[0]: coord[0] + 2 * size, coord[1]: coord[1] + 2 * size] = (lesion * - 1) + 1
            img = padded_img[size:padded_img.shape[0] - size, size:padded_img.shape[1] - size]
            
            img = minmax_scale(img.flatten(), feature_range = (intensities[i], 1)).reshape(struct_arr.shape[0], struct_arr.shape[1])
            
            lesion_img *= img
            ground_truth += (img * -1) + 1 
    
        else: 
            print("please provide lesion shape")
            
        mask = struct_arr > 0 
        lesion_img = lesion_img * mask
        ground_truth = ground_truth * mask
        
    return lesion_img, ground_truth


In [ ]:
def create_nonlinear_lesions(working_dir, dataset, num_lesions = 1, size = 5, intensity = 0.5, shape = 'circular', lesion_type = 'reduced', frac_pos = 0.5, both_lesion = False, window_type = None): 
    lesions_parts = []
    all_slices = []
    all_ground_truth = []
    all_labels = []

    participant_list = dataset['subj_idx']

    for part in participant_list: 
        print(part)

        path_data = os.path.join(working_dir, dataset['path'][part])
        scan = nib.load(path_data)
        struct_arr = scan.get_data().astype(np.float32)
        struct_arr = struct_arr[10:150, :, :135]
        slices = [struct_arr[:, :, i] for i in range(struct_arr.shape[2])]

        coords = []

        labels = np.zeros(len(slices))
        num_positives = int(frac_pos * len(slices))
        indices = np.arange(len(slices))

        both_lesions, separate_lesions = train_test_split(indices, test_size = 0.67)
        lesion_1, lesion_2 = train_test_split(separate_lesions, test_size = frac_pos)

        ground_truths = []

        for i, img in enumerate(slices): 
            if i in both_lesions: 
                coords = get_coordinates(slices[i], num_lesions = 2)
                lesion_img, ground_truth = create_simple_lesions(slices[i], list_coords = coords, size = size, 
                                                                intensities = intensities, shape = shape, lesion_type = lesion_type, window_type = window_type)

                slices[i] = lesion_img
                labels[i] = 1            
            elif i in lesion_1: 
                shape_1 = [shape[0]]
                coords = get_coordinates(slices[i], num_lesions = 1)
                lesion_img, ground_truth = create_simple_lesions(slices[i], list_coords = coords, size = size, 
                                                                intensities = [intensities[0]], shape = shape_1, lesion_type = [lesion_type[0]], window_type = window_type)

                slices[i] = lesion_img
                labels[i] = 0
            elif i in lesion_2:
                shape_2 = [shape[1]]
                coords = get_coordinates(slices[i], num_lesions = 1)
                lesion_img, ground_truth = create_simple_lesions(slices[i], list_coords = coords, size = size, 
                                                                intensities = [intensities[1]], shape = shape_2, lesion_type = [lesion_type[1]], window_type = window_type)

                slices[i] = lesion_img
                labels[i] = 0

            ground_truths.append((i, coords, ground_truth))

        all_slices.append(slices)
        all_ground_truth.append(ground_truths)
        all_labels.append(labels)

    return lesions_parts, all_slices, all_ground_truth, all_labels

## Create and save datasets with one type of lesions

In [ ]:
def create_lesions_slices(working_dir, dataset, num_lesions = 1, size = 5, intensity = 0.5, shape = 'circular', lesion_type = 'reduced', frac_pos = 0.5, both_lesion = False, mask = None, window_type = None, **kwargs):
    lesions_parts = []
    all_slices = []
    all_ground_truth = []
    all_labels = []
    
    participant_list = dataset['subj_idx']
    
    for part in participant_list: 
        print(part)
        
        path_data = os.path.join(working_dir, dataset['path'][part])
        scan = nib.load(path_data)
        struct_arr = scan.get_data().astype(np.float32)
        struct_arr = struct_arr[10:150, :, :135]
        slices = [struct_arr[:, :, i] for i in range(struct_arr.shape[2])]

        coords = []

        labels = np.zeros(len(slices))
        num_positives = int(frac_pos * len(slices))
        idx_pos = np.random.choice(len(slices), num_positives, replace = False)
        
        lesions_parts.append((part, idx_pos))

        ground_truths = []
        for i, img in enumerate(slices): 
            if i in idx_pos: 
                coords = get_coordinates(slices[i], num_lesions = num_lesions, mask = mask)
                lesion_img, ground_truth = create_simple_lesions(slices[i], list_coords = coords, size = size, 
                                                                intensities = intensity, shape = shape, lesion_type = lesion_type, window_type = window_type)

                slices[i] = lesion_img
                labels[i] = 1
            else: 
                if both_lesion: 
                    coords = get_coordinates(slices[i], num_lesions = kwargs['num_lesions_2'], mask = kwargs['mask_2'])
                    lesion_img, ground_truth = create_simple_lesions(slices[i], list_coords = coords, size = kwargs['size_2'], 
                                                                    intensities = kwargs['intensity_2'], shape = kwargs['shape_2'], lesion_type = kwargs['lesion_type_2'], window_type = kwards['window_type'])
                
                    slices[i] = lesion_img
                else: 
                    coords = None  
                    ground_truth = np.zeros((slices[i].shape[0], slices[i].shape[1]))
                
            ground_truths.append((i, coords, ground_truth))
        
        all_slices.append(slices)
        all_ground_truth.append(ground_truths)
        all_labels.append(labels)
    
    return lesions_parts, all_slices, all_ground_truth, all_labels 

## Generate data

In [ ]:
dataset = training_set
name = 'training'

In [ ]:
slice_ = deepcopy(slices[60])
mask = np.zeros((slice_.shape[0], slice_.shape[1]))
mask[:slice_.shape[0]//2, :slice_.shape[1]] = 1
mask_inverted = (mask * -1) + 1
plt.imshow(mask_inverted)
plt.colorbar()

In [ ]:
# params_group_2 = {'num_lesions_2' : 1, 'size_2' : 8, 'intensity_2' : [0.3], 'shape_2' : ['gaussian'], 'lesion_type_2' : [''], 'mask_2' : None}

In [ ]:
lesions_parts, all_slices, all_ground_truth, all_labels = create_lesions_slices(working_dir = working_dir, dataset = dataset, num_lesions = 1, shape = ['window'], intensity = [0.3], lesion_type = [''], size = 8, both_lesion = False, mask = mask, window_type = scipy.signal.hamming)

In [ ]:
fig = plt.figure(figsize = (10, 5))

plt.subplot(121)
plt.imshow(all_slices[0][50])

plt.subplot(122)
plt.imshow(all_ground_truth[0][50][2])

In [ ]:
fig = plt.figure(figsize = (10, 5))

plt.subplot(121)
plt.imshow(all_slices[0][52])

plt.subplot(122)
plt.imshow(all_ground_truth[0][52][2])

## Save data

In [ ]:
def save_data_pkl(array, data_dir, file_name):
    with open(os.path.join(data_dir, f'{file_name}.pkl'), 'wb') as f: 
        pkl.dump(array, f)
        
def save_data_h5py(list_data, list_labels, data_dir, file_name): 
    all_data = np.array(reduce(operator.add, list_data))
    all_labels = [label for labels_scan in list_labels for label in labels_scan]
    assert len(all_data) == len(all_labels)
    
    h5 = h5py.File(os.path.join(data_dir, f'{file_name}.h5'), 'w')
    h5.create_dataset('X', data=all_data, compression='gzip', compression_opts=9)
    h5.create_dataset('y', data=all_labels, compression='gzip', compression_opts=9)
    h5.close() 

In [ ]:
print("Starting at" + time.ctime())
start = time.time()

save_data_pkl(lesions_parts, data_dir = data_dir, file_name = f'lesion_locations_slices_{name}')
save_data_pkl(all_ground_truth, data_dir = data_dir, file_name = f'ground_truth_maps_{name}')
save_data_pkl(all_slices, data_dir = data_dir, file_name = f'slices_parts_{name}')
save_data_pkl(all_labels, data_dir = data_dir, file_name = f'labels_{name}')

end = time.time()
print("Runtime: " + str(datetime.timedelta(seconds = (end - start))))

In [ ]:
print("Starting at" + time.ctime())
start = time.time()

save_data_h5py(all_slices, all_labels, data_dir = data_dir, file_name = f'{name}_data')

end = time.time()
print("Runtime: " + str(datetime.timedelta(seconds = (end - start))))

In [ ]:
quit()